In [1]:
import numpy as np
import pandas as pd


path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv', parse_dates=["Timestamp"])
test = pd.read_csv(path + 'test_data.csv', parse_dates=["Timestamp"])

dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])


In [2]:
diff = dat.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
dat['solve_time'] = diff['Timestamp']

In [3]:
# 시험 난이도(?)로 추청되는 특성 따로 분류
dat['b_category'] = dat['assessmentItemID'].str[2]
# 시험지로 추청되는 특성 따로 분류. 뒤에 것이 좋을 것 같은데 일단 제출을 한 것으로 써놓음.
dat['test_category'] = dat['assessmentItemID'].str[4:7] # dat['assessmentItemID'].str[2] + dat['assessmentItemID'].str[4:7]
dat['problem_id'] = dat['assessmentItemID'].str[-3:]


In [4]:
diff['Timestamp'] = dat['solve_time']
diff['Timestamp'] = pd.qcut(diff['Timestamp'],5)
diff['Timestamp'] = diff['Timestamp'].astype("str")
dat['category_st_qcut_5'] = diff['Timestamp']

In [5]:
dat['solve_time'] = dat['solve_time'].astype(int)

In [6]:
dat['last_answerCode'] = dat.groupby("userID")['answerCode'].shift(1).fillna(1)


In [7]:
dat['last_answerCode'] = dat['last_answerCode'].astype(int)

In [8]:
dat['year'] = dat['Timestamp'].dt.year # 연도 정보
dat['month'] =  dat['Timestamp'].dt.month # 월 정보
dat['day'] =  dat['Timestamp'].dt.day # 일 정보
dat['hour'] =  dat['Timestamp'].dt.hour # 시간 정보

In [9]:
 #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
dat['user_correct_answer'] = dat.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
dat['user_correct_answer'] = dat['user_correct_answer'].fillna(int(0))
dat['user_correct_answer'] = dat['user_correct_answer'].astype(int)
dat['user_total_answer'] = dat.groupby('userID')['answerCode'].cumcount()
dat['user_acc'] = dat['user_correct_answer']/dat['user_total_answer']
dat['user_acc'] = dat['user_acc'].fillna(float(0))
# testId와 KnowledgeTag의 전체 정답률은 한번에 계산
# 아래 데이터는 제출용 데이터셋에 대해서도 재사용
correct_t = dat.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
correct_t.columns = ["test_mean", 'test_sum']
correct_k = dat.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
correct_k.columns = ["tag_mean", 'tag_sum']
dat = pd.merge(dat, correct_t, on=['testId'], how="left")
dat = pd.merge(dat, correct_k, on=['KnowledgeTag'], how="left")

In [17]:
dat['user_acc_5'] = pd.qcut(dat['user_acc'],5)
dat['user_acc_5'] = dat['user_acc_5'].astype("str")

dat['tag_mean_5'] = pd.qcut(dat['tag_mean'],5)
dat['tag_mean_5'] = dat['tag_mean_5'].astype("str")

dat['test_mean_5'] = pd.qcut(dat['test_mean'],5)
dat['test_mean_5'] = dat['test_mean_5'].astype("str")



tem3 = pd.DataFrame({'tag_mode' : dat.groupby('userID')['KnowledgeTag'].agg(pd.Series.mode)})
tem3['tag_mode'] = tem3['tag_mode'].apply(lambda x : x if str(type(x)) =="<class 'numpy.int64'>" else x[0])
tem3

# 유저별 한 문제 푸는 평균 시간, 총 문제 푼 시간 (초단위)

tem2 = dat.groupby('userID')['solve_time']
tem2 = pd.DataFrame({'time_mean' : tem2.mean(), 'time_sum':tem2.sum()}).reset_index()
tem2

# 유저 별 정답률, 푼 문제 개수

tem1 = dat.groupby('userID')['answerCode']
tem1 = pd.DataFrame({'answer_mean' : tem1.mean(), 'answer_cnt':tem1.count()}).reset_index()
tem1

# 유저별 최근 3개 문제의 정답률

tem4 = dat.groupby('userID')['answerCode'].apply(lambda x : x.iloc[-3:])
tem4 = pd.DataFrame({'last3_mean' : tem4.groupby('userID').mean()})
tem4

아직 추가 못함

In [18]:
dat

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,...,user_correct_answer,user_total_answer,user_acc,test_mean,test_sum,tag_mean,tag_sum,user_acc_5,tag_mean_5,test_mean_5
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,6,001,001,...,0,0,0.000000,0.952667,1429,0.957333,718,"(-0.001, 0.515]","(0.789, 0.973]","(0.791, 0.955]"
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3,6,001,002,...,1,1,1.000000,0.952667,1429,0.917067,3439,"(0.814, 1.0]","(0.789, 0.973]","(0.791, 0.955]"
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8,6,001,003,...,2,2,1.000000,0.952667,1429,0.917067,3439,"(0.814, 1.0]","(0.789, 0.973]","(0.791, 0.955]"
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7,6,001,004,...,3,3,1.000000,0.952667,1429,0.917067,3439,"(0.814, 1.0]","(0.789, 0.973]","(0.791, 0.955]"
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7,6,001,005,...,4,4,1.000000,0.952667,1429,0.917067,3439,"(0.814, 1.0]","(0.789, 0.973]","(0.791, 0.955]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24,3,071,005,...,1,4,0.250000,0.666000,999,0.694889,3127,"(-0.001, 0.515]","(0.614, 0.717]","(0.616, 0.714]"
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,4,165,001,...,1,5,0.200000,0.652500,783,0.698551,2410,"(-0.001, 0.515]","(0.614, 0.717]","(0.616, 0.714]"
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11,4,165,002,...,2,6,0.333333,0.652500,783,0.698551,2410,"(-0.001, 0.515]","(0.614, 0.717]","(0.616, 0.714]"
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46,4,165,003,...,3,7,0.428571,0.652500,783,0.698551,2410,"(-0.001, 0.515]","(0.614, 0.717]","(0.616, 0.714]"


In [19]:
dat.isnull().sum()

userID                 0
assessmentItemID       0
testId                 0
answerCode             0
Timestamp              0
KnowledgeTag           0
solve_time             0
b_category             0
test_category          0
problem_id             0
category_st_qcut_5     0
last_answerCode        0
year                   0
month                  0
day                    0
hour                   0
user_correct_answer    0
user_total_answer      0
user_acc               0
test_mean              0
test_sum               0
tag_mean               0
tag_sum                0
user_acc_5             0
tag_mean_5             0
test_mean_5            0
dtype: int64

In [20]:
dat.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'solve_time', 'b_category', 'test_category',
       'problem_id', 'category_st_qcut_5', 'last_answerCode', 'year', 'month',
       'day', 'hour', 'user_correct_answer', 'user_total_answer', 'user_acc',
       'test_mean', 'test_sum', 'tag_mean', 'tag_sum', 'user_acc_5',
       'tag_mean_5', 'test_mean_5'],
      dtype='object')

In [21]:
dat.to_csv('/opt/ml/input/data/FE/FE_total.csv', index=False)